# 데이터 과제 - Pandas 활용 연습

In [1]:
import pandas as pd

orders_data = {
    'order_id': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'user_id': [1, 2, 1, 3, 2, 3, 4, 4, 5, 1],
    'product': ['TV', 'Laptop', 'TV', 'Mouse', 'Keyboard', 'Mouse', 'Phone', 'Phone', 'TV', 'Tablet'],
    'price': [800000, 1200000, 750000, 50000, 100000, 45000, 900000, 950000, 700000, 300000],
    'order_date': ['2024-12-01', '2024-12-03', '2025-01-15', '2025-01-20', '2025-02-10', 
                   '2025-02-11', '2025-03-01', '2025-03-05', '2025-03-10', '2025-04-01'],
    'category': ['Electronics', 'Electronics', 'Electronics', 'Accessories', 'Accessories', 
                 'Accessories', 'Electronics', 'Electronics', 'Electronics', 'Electronics']
}
orders_df = pd.DataFrame(orders_data)
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

users_data = {
    'user_id': [1, 2, 3, 4, 5],
    'age': [25, 32, 40, 28, 36],
    'gender': ['남', '여', '남', '여', '남']
}
users_df = pd.DataFrame(users_data)

## 📄 공통 데이터: `orders.csv`, `users.csv`

---

### ✅ 문제 1. 데이터 불러오기 및 전처리
1-1. `orders.csv` 파일을 불러오고, `order_date` 컬럼을 datetime 형식으로 변환하시오.  
1-2. 결측치가 있는 행은 제거하고, 총 몇 건의 데이터가 남았는지 출력하시오.

In [ ]:
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
orders_df.dropna(inplace=True)

print('남은 데이터 개수 : ', len(orders_df))

남은 데이터 개수 :  10


---

### ✅ 문제 2. 월별 매출 분석
2-1. 월(`YYYY-MM`) 단위로 그룹화하여 매출(`price`) 합계를 계산하시오.  
2-2. 가장 매출이 높았던 달은 언제이며, 매출은 얼마인가?

In [ ]:
orders_df['order_month'] = orders_df['order_date'].dt.to_period('M')
monthly_sales = orders_df.groupby('order_month')['price'].sum().sort_values(ascending=False)
print("가장 매출 높은 달:", monthly_sales.idxmax(), "매출:", monthly_sales.max())

         order_id  user_id    price
monthly                            
2024-12       203        3  2000000
2025-01       207        4   800000
2025-02       211        5   145000
2025-03       324       13  2550000
2025-04       110        1   300000
가장 매출 높은 달: 2025-03 매출: 2550000


---

### ✅ 문제 3. 사용자 분석
3-1. 사용자별 평균 주문 금액을 계산하고, 상위 5명을 출력하시오.  
3-2. 사용자별 주문 횟수를 구하고, 가장 주문을 많이 한 사용자의 ID와 주문 수를 출력하시오.

In [35]:
user_avg_price = orders_df.groupby('user_id')['price'].mean().sort_values(ascending=False).head(5)
print("상위 5명 평균 주문 금액:\n", user_avg_price)

상위 5명 평균 주문 금액:
 user_id
4    925000.000000
5    700000.000000
2    650000.000000
1    616666.666667
3     47500.000000
Name: price, dtype: float64


In [36]:
user_order_count = orders_df['user_id'].value_counts()
top_user = user_order_count.idxmax()
print("가장 주문 많이 한 사용자:", top_user, "주문 수:", user_order_count.max())

가장 주문 많이 한 사용자: 1 주문 수: 3


---

### ✅ 문제 4. 카테고리별 인기 상품 분석
4-1. 카테고리별로 주문량이 가장 많은 상품명을 출력하시오.

In [40]:
cat_order_count = orders_df['category'].value_counts()
print("가장 주문이 많은 카테고리 : ", cat_order_count.idxmax())

가장 주문이 많은 카테고리 :  Electronics


In [63]:
orders_df.groupby(['category', 'product']).size().reset_index(name='count').sort_values('count', ascending=False).groupby('category').first()

,product,count
category,,
Accessories,Mouse,2
Electronics,TV,3


In [52]:
top_products = orders_df.groupby(['category', 'product']).size().reset_index(name='count')
top_per_category = top_products.sort_values('count', ascending=False).groupby('category').first()
print("카테고리별 인기 상품:\n", top_per_category)

카테고리별 인기 상품:
             product  count
category                  
Accessories   Mouse      2
Electronics      TV      3


---

### ✅ 문제 5. 사용자 정보와 병합 (merge)
5-1. `orders.csv`와 `users.csv`를 `user_id` 기준으로 병합하시오.  
5-2. 성별(`gender`)별 총 주문 금액의 평균을 계산하시오.

In [4]:
merged = pd.merge(orders_df, users_df, on='user_id')

merged['order_month'] = merged['order_date'].dt.to_period('M')
pivot_sales = pd.pivot_table(
    merged,
    values='price',
    index='order_month',
    columns='gender',
    aggfunc='sum'
)
print(pivot_sales)

gender              남          여
order_month                     
2024-12      800000.0  1200000.0
2025-01      800000.0        NaN
2025-02       45000.0   100000.0
2025-03      700000.0  1850000.0
2025-04      300000.0        NaN


In [5]:
max_value = pivot_sales.max().max()
month_gender = pivot_sales.stack().idxmax()
print("최고 매출 조합:", month_gender, "매출:", max_value)

최고 매출 조합: (Period('2025-03', 'M'), '여') 매출: 1850000.0


---

### ✅ 문제 6. 피벗 테이블 활용
6-1. 월별(`YYYY-MM`)과 성별(`gender`)로 구분된 매출 합계를 피벗 테이블로 생성하시오.  
6-2. 매출이 가장 높았던 성별-월 조합과 해당 매출을 출력하시오.

In [ ]:
users = pd.read_csv('users.csv')
merged = pd.merge(df, users, on='user_id')
print("병합 후 데이터 수:", len(merged))

gender_avg_price = merged.groupby('gender')['price'].mean()
print("성별 평균 주문 금액:\n", gender_avg_price)

---

### ✅ 문제 7. 사용자 등급 분류 (apply)
7-1. 사용자별 평균 주문 금액을 기준으로 아래 기준에 따라 등급을 부여하시오:  
- 평균 주문금액 ≥ 100,000원: `'VIP'`  
- 평균 주문금액 ≥ 50,000원: `'Gold'`  
- 평균 주문금액 < 50,000원: `'Standard'`

7-2. 등급이 포함된 테이블을 `user_id`, `grade` 컬럼만 남겨 출력하시오.

---